In [12]:
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm


In [13]:
data = pd.read_parquet("data/full_qanun_text.parquet")
transformer = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-tas-b").to("cuda")

In [14]:
data.shape

(1501973, 1)

In [4]:
batch_size = 10_000
start = 0
indexes_list = []

In [5]:
for i in tqdm(range(0, 20000, batch_size)):    
    if i <= len(data) - batch_size:
        try:
            dt = list(data.iloc[start:i, :]["text"])
            dt = list(transformer.encode(dt))
            indexes_list += dt
            start += batch_size
        except IndexError as ex:
            print(ex)
            continue
    else:
        dt = list(data.iloc[i::, :]["text"])
        dt = list(transformer.encode(dt))
        indexes_list += dt
        start += batch_size

  0%|          | 0/2 [00:00<?, ?it/s]

list index out of range


100%|██████████| 2/2 [00:14<00:00,  7.30s/it]


In [5]:
len(indexes_list[0])

768

In [6]:
indexes_list = np.array(indexes_list)

In [9]:
len(indexes_list)

1491973

In [8]:
index = faiss.index_factory(768, 'IVF15,Flat', faiss.METRIC_INNER_PRODUCT)
faiss.normalize_L2(indexes_list)
index.train(indexes_list)
index.add(indexes_list)
faiss.write_index(index, "trained_index.index")

In [9]:
index.nprobe = 5

In [10]:
def search(request:str):
    vector = np.array([transformer.encode(request)])
    faiss.normalize_L2(vector)
    D, I = index.search(vector, 15)
    return D, I

In [11]:
search("Hello world")

(array([[0.7321978 , 0.7301693 , 0.7290461 , 0.7290461 , 0.7288023 ,
         0.7275504 , 0.7257741 , 0.7256943 , 0.7253185 , 0.7252851 ,
         0.72511405, 0.725114  , 0.72483623, 0.72442496, 0.7242232 ]],
       dtype=float32),
 array([[3521, 3482, 3147, 3067, 3483, 3507, 3340, 2493, 3608, 2746, 3534,
         3503,  735, 3538, 9994]], dtype=int64))